In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [5]:
#Load in the data

df_calendar = pd.read_csv('/Users/Caddi/Data/BerlinAirbnb/calendar.csv')
df_listings = pd.read_csv('/Users/Caddi/Data/BerlinAirbnb/listings.csv')
df_reviews = pd.read_csv('/Users/Caddi/Data/BerlinAirbnb/reviews.csv')

In a first step, we will explore the datasets. 